In [100]:
# Trying to understand the ordering of basis states in even-electron SO-CI (Molpro)
# 6/7/2023
import re, sys, copy, glob, os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
sys.path.insert(0, '../atomic_SOC')
import molpro_subs as mpr
import chem_subs as chem

pd.set_option('display.width', 1000)
pd.set_option('display.max_rows', 500)

In [101]:
fsoci = 'ac5z_hybB_r2p2444_lz.pro'  # PtH
fsoci = 'ac5z_35D15Q_r1p5400_lz.pro'  # PtH
#fsoci = 'ac5z_soc2011_r1p5987_21S15T_lz.pro' # PtH-
#fsoci = '../UMemphis/mgcl-equil-karl-block.pro'
fsoci = '../UMemphis/oh.pro'
fsoci = '../UMemphis/mgcl_small4.pro'
print('Will read SO-CI info from file: {:s}'.format(fsoci))

Will read SO-CI info from file: ../UMemphis/mgcl_small4.pro


In [102]:
# Read CASSCF
PG = mpr.read_point_group(fsoci)
print('Point group is', PG)
crd, lineno_crd = mpr.read_coordinates(fsoci, linenum=True)
if isinstance(lineno_crd, list):
    # take last geometry
    crd = crd[-1]
    lineno_crd = lineno_crd[-1]
# get diatomic bond length
G = chem.Geometry(crd, intype='DataFrame', units='bohr')
G.toAngstrom()
R = np.round(G.distance(0, 1), 6)  # round the bond length to 6 digits
print('Bond length = {:.4f}'.format(R))
caslist, lineno_cas = mpr.readMULTI(fsoci, PG=PG, linenum=True)
CAS = caslist[-1]   # assume the last CASSCF to be the relevant one

oldcas = CAS.results.copy()
for Spin in sorted(set(oldcas.Spin)):
    nspin = len(oldcas[oldcas.Spin == Spin])
    print('{:d} {:s}s'.format(nspin, Spin))
print('Active space = {:d}/{:d}'.format(CAS.nactel(), CAS.nactorb()))
CAS.results = mpr.relabel_CAS_by_energy(CAS.results)
# Note any changes in CAS labeling
diflbl = (oldcas.Label != CAS.results.Label).values
if diflbl.any():
    dflbl = oldcas[['Label', 'Term']].copy()
    dflbl['NewLabel'] = CAS.results.Label
    print('Some CAS labels changed:')
    display(dflbl)
# check for dynamical weights
rx_dynw = re.compile('[~!]*dynw,(\d+)')
dynw = 0
with open(fsoci, 'r') as F:
    for line in F:
        m = rx_dynw.search(line)
        if m:
            dynw = int(m.group(1))
            print('Dynamical weighting with dynw = {:d}'.format(dynw))
if not dynw:
    print('Uniform weighting')

Point group is C2v
Bond length = 2.3000
3 Doublets
Active space = 9/8
Uniform weighting


In [103]:
# read MRCI
cilist, lineno_ci = mpr.readMRCI(fsoci, linenum=True)   # probably many
for m in cilist:
    m.transfer_lz(CAS.results)
mrci = [mpr.MRCIstate(row) for m in cilist for (irow, row) in m.results.iterrows()]
dfci = mpr.combineMRCI(cilist).drop(columns=['Ncore', 'dipX', 'dipY', 'Dipole', 'Configs'])
dfci[['Spin', 'Label', 'Energy']]
nci = len(dfci)  # number of MRCI states
print(f'{nci} MRCI states')
dfci

3 MRCI states


,Group,Spin,Irrep,Label,Energy,Edav,dipZ,Eref,Ref,C0,Lz,Term,record
0,1,Doublet,1,1.1,-661.143294,-661.160817,-1.498847,-660.933350,1.1,0.96071,0.0,2Σ+,5101.2
1,2,Doublet,2,1.2,-661.022124,-661.040521,-1.854241,-660.810538,1.2,0.95917,1.0,2Π,5201.2
2,3,Doublet,3,1.3,-661.022124,-661.040521,-1.854241,-660.810538,1.3,0.95917,1.0,2Π,5301.2


In [104]:
print('Checking "Wavefunction restored" lines for consistency with MRCI')
re_start = re.compile('Spin-orbit matrix elements')
re_end = re.compile('Spin-orbit calculation in the basis of zeroth order wave functions')
re_record = re.compile(r'\s*Wavefunction restored from record')
re_eci = re.compile(' Original energies: ')
loadrec = []  # list of record numbers
inblock = False
with open(fsoci) as F:
    for line in F:
        if not inblock:
            if re_start.search(line):
                inblock = True
            else:
                continue
        # reach this point only if in the section
        if re_record.match(line):
            # check that this info is consistent with the MRCI
            #print(line.rstrip())
            w = line.split()
            record = w[4]
            irrep = int(w[5].split('=')[1])
            S = float(w[7])
            nstate = int(w[-1])
            if record not in dfci.record.values:
                print(f'*** Unknown record number {record}')
            else:
                dfrec = dfci[dfci.record == record]
                if nstate != len(dfrec):
                    print(f'*** NSTATE = {nstate} but there are {len(dfrec)} MRCI states')
                irr = set(dfrec.Irrep.values).pop()
                if irrep != irr:
                    print(f'*** Symmetry={irrep} but it is {irr} in the MRCI')
                sp = chem.MULTSPIN[set(dfrec.Spin.values).pop()]
                if sp != S:
                    print(f'*** S= {S} but it is {sp} in the MRCI')
        if re_eci.match(line):
            eci = np.array(line.split()[2:]).astype(float) # threse are printed to 6 digits
            emrci = np.round(dfci[dfci.record == record].Energy.values, 6)
            ok = True
            for eread, ewrote in zip(eci, emrci):
                if abs(eread - ewrote) > 1.e-6:
                    ok = False
            if not ok:
                print(f'Inconsistent CI energies from record {record}')
                display(pd.DataFrame({'written': emrci, 'read': eci}))
        if re_end.search(line):
            inblock = False

Checking "Wavefunction restored" lines for consistency with MRCI


In [105]:
print('Reading components of SO matrix elements')
print('\twith attention to both systems of labeling the MRCI states')
# The CI state numbers here are ordinal and may differ from those in the MRCI section of output
re_bra = re.compile(' Bra-wavefunction restored from record ')
re_ket = re.compile(' Ket-wavefunction restored from record ')
re_elem = re.compile(' !MRCI trans\s+<(\d+\.\d)\|(?:ECP)?(LS[XYZ])\|(\d+\.\d)>')
lsmat = {}
for op in ['LSX', 'LSY', 'LSZ']:
    lsmat[op] = np.zeros((nci, nci), dtype=complex)
oldCI = [] # list of CI state labels as printed in the SO-CI section
newCI = [] # list of corresponding MRCI state labels from the MRCI section
Ss = []  # list of S values for CI states
icis = []  # list of MRCI state numbers (index into dfci)
with open(fsoci) as F:
    for line in F:
        if re_bra.match(line):
            recbra = line.split()[-1]
        if re_ket.match(line):
            recket = line.split()[-1]
        m = re_elem.match(line)
        if m:
            cibra = m.group(1)
            op = m.group(2)
            ciket = m.group(3)
            elem = line.split()[3]
            print(f'<{recbra}({cibra})|{op}|{recket}({ciket})> = {elem}', end='')
            # these MRCI state numbers (cibra and ciket) are ordinal (in dfci)
            #   get the correct labels from the MRCI section
            c = complex(elem.replace('i', 'j'))
            ibra = int(cibra.split('.')[0]) - 1
            iket = int(ciket.split('.')[0]) - 1
            dfbra = dfci[dfci.record == recbra]
            dfket = dfci[dfci.record == recket]
            lblbra = dfbra.Label.values[ibra]
            lblket = dfket.Label.values[iket]
            print(f' --> <{lblbra}|{op}|{lblket}> = {c}')
            I = dfbra.index[ibra]  # index into dfci
            J = dfket.index[iket]
            if lsmat[op][I, J] != 0:
                # Two values are provied in non-ECP case; prefer the second value (Breit-Pauli)
                print(f'\tReplacing previous matrix element: {lsmat[op][I, J]}')
            lsmat[op][I, J] = c
            # make Hermitian
            lsmat[op][J, I] = np.conj(c)
            # keep track of both systems for labeling the CI states
            oldCI.extend([cibra, ciket])
            newCI.extend([lblbra, lblket])
            icis.extend([I, J])
            Sbra = chem.MULTSPIN[dfbra.Spin.values[ibra]]
            Sket = chem.MULTSPIN[dfket.Spin.values[iket]]
            Ss.extend([Sbra, Sket])
lsmat['tot'] = lsmat['LSX'] + lsmat['LSY'] + lsmat['LSZ']  # this is for MRCI states, not BSs
dfCIlabels = pd.DataFrame({'CIbad': oldCI, 'CIgood': newCI, 'S': Ss, 'ici': icis }).drop_duplicates(ignore_index=True)

Reading components of SO matrix elements
	with attention to both systems of labeling the MRCI states
<5101.2(1.1)|LSX|5301.2(1.3)> = -0.000129557035i --> <1.1|LSX|1.3> = -0.000129557035j
<5101.2(1.1)|LSX|5301.2(1.3)> = -0.000129522521i --> <1.1|LSX|1.3> = -0.000129522521j
	Replacing previous matrix element: -0.000129557035j
<5101.2(1.1)|LSY|5201.2(1.2)> = -0.000129557049 --> <1.1|LSY|1.2> = (-0.000129557049+0j)
<5101.2(1.1)|LSY|5201.2(1.2)> = -0.000129522535 --> <1.1|LSY|1.2> = (-0.000129522535+0j)
	Replacing previous matrix element: (-0.000129557049+0j)
<5201.2(1.2)|LSZ|5301.2(1.3)> = -0.000100907776i --> <1.2|LSZ|1.3> = -0.000100907776j
<5201.2(1.2)|LSZ|5301.2(1.3)> = -0.000100826433i --> <1.2|LSZ|1.3> = -0.000100826433j
	Replacing previous matrix element: -0.000100907776j


In [106]:
print('Reading the SO state energies')
SOe = mpr.readSOenergy(fsoci)
dimen = len(SOe.results)
print(f'\t{dimen} SO states')

Reading the SO state energies
	6 SO states


In [107]:
print('Reading the SO matrix as diplayed together')
soblock, lineno = mpr.readSOmatrixBlocks(fsoci)
SOmat, BSlbls, OKlbls = mpr.parse_SOmatrix(soblock[0], dimen)
if not OKlbls:
    # BS labels require repair
    ok = True
    oldlbls = BSlbls
    BSlbls = mpr.fix_SObasis(oldlbls, SOmat, mrci)
    # Check for consistency with the earlier label mapping
    for bbad, bgood in zip(oldlbls, BSlbls):
        for irow, row in dfCIlabels.iterrows():
            if (row.CIgood == bgood[0]) and (row.S == bgood[1]):
                # the unique "good" labels match
                if (row.CIbad == bbad[0]) and (row.S == bbad[1]):
                    # the repeated "bad" labels also match, as they should
                    pass
                else:
                    # not consistent!
                    print('*** MRCI and BS label maps disagree!')
                    ok = False
    if ok:
        print('BS label re-mapping was successful')
else:
    print('BS labels are fine without correction')
# Get the parent MRCI state for each BS
icis = []
for BSlbl in BSlbls:
    # find the matching MRCI state
    subdf = dfCIlabels[(dfCIlabels.CIgood == BSlbl[0]) & (dfCIlabels.S == BSlbl[1])]
    icis.append(subdf.ici.values[0]) # there should be only one value
if not OKlbls:
    dfBS = pd.DataFrame({'BSbad': oldlbls, 'BSgood': BSlbls, 'ici': icis})
else:
    dfBS = pd.DataFrame({'BSgood': BSlbls, 'ici': icis})
display(dfBS)

Reading the SO matrix as diplayed together
BS labels are fine without correction


,BSgood,ici
0,"(1.1, 0.5, 0.5)",0
1,"(1.1, 0.5, -0.5)",0
2,"(1.2, 0.5, 0.5)",1
3,"(1.2, 0.5, -0.5)",1
4,"(1.3, 0.5, 0.5)",2
5,"(1.3, 0.5, -0.5)",2


In [108]:
# convert components from hartree to cm-1 units
for m in lsmat.values():
    m *= chem.AU2CM

In [110]:
def smat_elem(S1, M1, S2, M2, matchS=True):
    # return (<Sx>, <Sy>, <Sz>)
    # if matchS==True, then return 0 if S1 != S2
    print(S1, M1, S2, M2)
    if matchS and (S1 != S2):
        return 0j, 0j, 0j
    elz = elplus = elminus = 0
    # <Sz>
    if M1 == M2:
        elz = M2
    # <S+>
    if M1 == (M2 + 1):
        elplus = np.sqrt((S2*(S2+1) - M2*M1))
    # <S->
    if M1 == (M2 - 1):
        elminus = np.sqrt((S2*(S2+1) - M2*M1))
    print('+/-:', elplus, elminus)
    # Sx = (1/2)(S+ + S-); Sy = (-i/2)(S+ - S-)
    elx = (1/2) * (elplus + elminus)
    ely = (1/2j) * (elplus - elminus)
    return elx, ely, elz

In [111]:
i = 3
j = 5
ibas = dfBS.at[i, 'BSgood']
jbas = dfBS.at[j, 'BSgood']
print(ibas, jbas)
smat_elem(ibas[1], ibas[2], jbas[1], jbas[2])

('1.2', 0.5, -0.5) ('1.3', 0.5, -0.5)
0.5 -0.5 0.5 -0.5
+/-: 0 0


(0.0, 0j, -0.5)

In [112]:
# Attempt to compute matrix elements of Sx, Sy, Sz
smat = {}
dimen = len(dfBS)
for i in ['Sx', 'Sy', 'Sz', 'total']:
    smat[i] = np.zeros((dimen, dimen), dtype=complex)
#  DataFrame for displaying results
data = {}
for col in ['ibas', 'jbas', 'Lx', 'Ly', 'Lz', 'Sx', 'Sy', 'Sz', 'total', 'displayed']:
    data[col] = []
for ibas, irow in dfBS.iterrows():
    S1 = irow.BSgood[1]
    M1 = irow.BSgood[2]
    ici = irow.ici
    for jbas, jrow in dfBS.iterrows():
        data['ibas'].append(ibas)
        data['jbas'].append(jbas)
        S2 = jrow.BSgood[1]
        M2 = jrow.BSgood[2]
        jci = jrow.ici
        # <Sz>
        if (S1 == S2) and (M1 == M2):
            elz = M2
        else:
            elz = 0
        smat['Sz'][ibas, jbas] = elz
        # <S+>
        if (S1 == S2) and (M1 == M2 + 1):
            elplus = np.sqrt((S2*(S2+1) - M2*(M2+1)))
        else:
            elplus = 0
        # <S->
        if (S1 == S2) and (M1 == M2 - 1):
            elminus = np.sqrt((S2*(S2+1) - M2*(M2-1)))
        else:
            elminus = 0
        # Sx = (1/2)(S+ + S-); Sy = (-i/2)(S+ - S-)
        elx = (1/2) * (elplus + elminus)
        ely = (-1j / 2) * (elplus - elminus)
        smat['Sx'][ibas, jbas] = elx
        smat['Sy'][ibas, jbas] = ely
        # get the spatial part from the CI matrix element (during computation step)
        x = lsmat['LSX'][ici, jci]
        y = lsmat['LSY'][ici, jci]
        z = lsmat['LSZ'][ici, jci]
        # total matrix element as dot product of spatial * spin
        eltot = x * elx + y * ely + z * elz 
        smat['total'][ibas, jbas] = eltot
        data['Lx'].append(x)
        data['Ly'].append(y)
        data['Lz'].append(z)
        data['Sx'].append(elx)
        data['Sy'].append(ely)
        data['Sz'].append(elz)
        data['total'].append(np.round(eltot, 2))
        data['displayed'].append(SOmat[ibas, jbas])
dfshow = pd.DataFrame(data)

In [115]:
display(dfBS)
def ferr(mine, its):
    # simple error code
    a = np.round(mine, 2)
    b = np.round(its, 2)
    if a == b:
        return 0
    if a == 0:
        return b
    return np.round(b / a, 2)
dfshow['err'] = [ferr(mine, its) for mine, its in zip(dfshow.total, dfshow.displayed)]
dfshow[(dfshow.err != 0) & (dfshow.ibas != dfshow.jbas)].sort_values('err')

,BSgood,ici
0,"(1.1, 0.5, 0.5)",0
1,"(1.1, 0.5, -0.5)",0
2,"(1.2, 0.5, 0.5)",1
3,"(1.2, 0.5, -0.5)",1
4,"(1.3, 0.5, 0.5)",2
5,"(1.3, 0.5, -0.5)",2


,ibas,jbas,Lx,Ly,Lz,Sx,Sy,Sz,total,displayed,err
3,0,3,0.000000+0.000000j,-28.426911+0.000000j,0.000000+0.000000j,0.5,0.0-0.5j,0.0,0.00+14.21j,28.43+0.00j,0.0-2.0j
8,1,2,0.000000+0.000000j,-28.426911+0.000000j,0.000000+0.000000j,0.5,0.0+0.5j,0.0,0.00-14.21j,-28.43+0.00j,-0.0-2.0j
13,2,1,0.000000+0.000000j,-28.426911-0.000000j,0.000000+0.000000j,0.5,0.0-0.5j,0.0,0.00+14.21j,-28.43+0.00j,0.0+2.0j
18,3,0,0.000000+0.000000j,-28.426911-0.000000j,0.000000+0.000000j,0.5,0.0+0.5j,0.0,0.00-14.21j,28.43+0.00j,-0.0+2.0j
5,0,5,0.000000-28.426908j,0.000000+0.000000j,0.000000+0.000000j,0.5,0.0-0.5j,0.0,0.00-14.21j,0.00-28.43j,2.0-0.0j
10,1,4,0.000000-28.426908j,0.000000+0.000000j,0.000000+0.000000j,0.5,0.0+0.5j,0.0,0.00-14.21j,0.00-28.43j,2.0-0.0j
16,2,4,0.000000+0.000000j,0.000000+0.000000j,0.000000-22.128844j,0.0,0.0-0.0j,0.5,0.00-11.06j,0.00-22.13j,2.0-0.0j
23,3,5,0.000000+0.000000j,0.000000+0.000000j,0.000000-22.128844j,0.0,0.0-0.0j,-0.5,0.00+11.06j,0.00+22.13j,2.0+0.0j
25,4,1,0.000000+28.426908j,0.000000+0.000000j,0.000000+0.000000j,0.5,0.0-0.5j,0.0,0.00+14.21j,0.00+28.43j,2.0+0.0j
26,4,2,0.000000+0.000000j,0.000000+0.000000j,0.000000+22.128844j,0.0,0.0-0.0j,0.5,0.00+11.06j,0.00+22.13j,2.0+0.0j


In [72]:
dfshow[((dfshow.total == 0) | (dfshow.displayed == 0)) & (dfshow.total != dfshow.displayed)]

,ibas,jbas,Lx,Ly,Lz,Sx,Sy,Sz,total,displayed
14,2,2,0.0+0.0j,0.0+0.0j,0.0+0.0j,0.0,0.0-0.0j,0.5,0.0+0.0j,26401.9+0.0j
21,3,3,0.0+0.0j,0.0+0.0j,0.0+0.0j,0.0,0.0-0.0j,-0.5,0.0+0.0j,26401.9+0.0j
28,4,4,0.0+0.0j,0.0+0.0j,0.0+0.0j,0.0,0.0-0.0j,0.5,0.0+0.0j,26401.9+0.0j
35,5,5,0.0+0.0j,0.0+0.0j,0.0+0.0j,0.0,0.0-0.0j,-0.5,0.0+0.0j,26401.9+0.0j


In [62]:
for i in range(dimen):
    for j in range(dimen):
        print('{:<3d} {:<3d}  {:12.5f}  \t|  {:12.5f}'.format(i,j, smat['total'][i, j], SOmat[i, j]))

0   0    0.00000+0.00000j  	|  0.00000+0.00000j
0   1    0.00000+0.00000j  	|  0.00000+0.00000j
0   2    0.00000+0.00000j  	|  0.00000+0.00000j
0   3    0.00000+14.21346j  	|  28.43000+0.00000j
0   4    0.00000+0.00000j  	|  0.00000+0.00000j
0   5    0.00000-14.21345j  	|  0.00000-28.43000j
1   0    0.00000+0.00000j  	|  0.00000+0.00000j
1   1    0.00000+0.00000j  	|  0.00000+0.00000j
1   2    0.00000-14.21346j  	|  -28.43000+0.00000j
1   3    0.00000+0.00000j  	|  0.00000+0.00000j
1   4    0.00000-14.21345j  	|  0.00000-28.43000j
1   5    0.00000+0.00000j  	|  0.00000+0.00000j
2   0    0.00000+0.00000j  	|  0.00000+0.00000j
2   1    0.00000+14.21346j  	|  -28.43000+0.00000j
2   2    0.00000+0.00000j  	|  26401.90000+0.00000j
2   3    0.00000+0.00000j  	|  0.00000+0.00000j
2   4    0.00000-11.06442j  	|  0.00000-22.13000j
2   5    0.00000+0.00000j  	|  0.00000+0.00000j
3   0    0.00000-14.21346j  	|  28.43000+0.00000j
3   1    0.00000+0.00000j  	|  0.00000+0.00000j
3   2    0.00000+0.0

In [42]:
smat['total']

array([[0. -0.j        , 0. -0.j        , 0.+33.71045072j,
        0. -0.j        ],
       [0. +0.j        , 0. -0.j        , 0. -0.j        ,
        0.-33.71045072j],
       [0.-33.71045072j, 0. +0.j        , 0. -0.j        ,
        0. -0.j        ],
       [0. +0.j        , 0.+33.71045072j, 0. +0.j        ,
        0. -0.j        ]])

In [43]:
SOmat

array([[0. +0.j  , 0. +0.j  , 0.+67.42j, 0. +0.j  ],
       [0. +0.j  , 0. +0.j  , 0. +0.j  , 0.-67.42j],
       [0.-67.42j, 0. +0.j  , 0. +0.j  , 0. +0.j  ],
       [0. +0.j  , 0.+67.42j, 0. +0.j  , 0. +0.j  ]])

In [12]:
print('Comparing components of non-zero matrix elements with displayed matrix')
ij = []
elrep = []
elsum = []
lsx = []
lsy = []
lsz = []
Scomp = []
Szcomp = []
Lzcomp = []
bsi = []
bsj = []
adiff = []
asum = []
matsum = np.zeros_like(SOmat)
for ibas in range(dimen):
    ici = dfBS.at[ibas, 'ici']
    matsum[ibas, ibas] = (dfci.at[ici, 'Edav'] - dfci.Edav.min()) * chem.AU2CM
    for jbas in range(ibas):
        elem = SOmat[ibas, jbas]
        jci = dfBS.at[jbas, 'ici']
        x = lsmat['LSX'][ici, jci]
        y = lsmat['LSY'][ici, jci]
        z = lsmat['LSZ'][ici, jci]
        tot = lsmat['tot'][ici, jci]
        matsum[ibas, jbas] = tot
        matsum[jbas, ibas] = np.conj(tot)
        elrep.append(elem)
        lsx.append(np.round(x, 2))
        lsy.append(np.round(y, 2))
        lsz.append(np.round(z, 2))
        elsum.append(np.round(tot, 2))
        adiff.append(np.absolute(elem - tot))
        asum.append(np.absolute(elem + tot))
        bsi.append(BSlbls[ibas])
        bsj.append(BSlbls[jbas])
        ij.append([ici, jci])
        # try to find the missing sign!
        Sbra = dfBS.at[ibas, 'BSgood'][1]
        Sket = dfBS.at[jbas, 'BSgood'][1]
        szbra = dfBS.at[ibas, 'BSgood'][2]
        szket = dfBS.at[jbas, 'BSgood'][2]
        Scomp.append(f'{Sbra}|{Sket}')
        Szcomp.append(f'({szbra}, {szket})')
        Lzcomp.append(f'({dfci.at[ici, "Lz"]}, {dfci.at[jci, "Lz"]})')
        if (np.absolute(elem) < 0.01) and (np.absolute(tot) < 0.01):
            # don't display zeros that agree
            continue
        # round for display
        x = np.round(x, 2)
        y = np.round(y, 2)
        z = np.round(z, 2)
        tot = np.round(tot, 2)
dfelemcomp = pd.DataFrame({'(i,j)': ij, '(Szi, Szj)': Szcomp, '(Lzi, Lzj)': Lzcomp,
                           'displayed': elrep, 'summed': elsum,
                           'BSi': bsi, 'BSj': bsj,
                           '<LSX>': lsx, '<LSY>': lsy, '<LSZ>': lsz})
fmt = {}
for col in ['summed', 'displayed', 'LSX', 'LSY', 'LSZ']:
    fmt[col] = '{:.2f}'
display(dfelemcomp[(dfelemcomp.displayed != 0) | (dfelemcomp.summed != 0)])

Comparing components of non-zero matrix elements with displayed matrix


,"(i,j)","(Szi, Szj)","(Lzi, Lzj)",displayed,summed,BSi,BSj,<LSX>,<LSY>,<LSZ>
1,"[1, 0]","(0.5, 0.5)","(1.0, 1.0)",0.00-67.42j,0.00-67.42j,"(1.3, 0.5, 0.5)","(1.2, 0.5, 0.5)",0.0+0.0j,0.0+0.0j,0.00-67.42j
2,"[1, 0]","(0.5, -0.5)","(1.0, 1.0)",0.00+0.00j,0.00-67.42j,"(1.3, 0.5, 0.5)","(1.2, 0.5, -0.5)",0.0+0.0j,0.0+0.0j,0.00-67.42j
3,"[1, 0]","(-0.5, 0.5)","(1.0, 1.0)",0.00+0.00j,0.00-67.42j,"(1.3, 0.5, -0.5)","(1.2, 0.5, 0.5)",0.0+0.0j,0.0+0.0j,0.00-67.42j
4,"[1, 0]","(-0.5, -0.5)","(1.0, 1.0)",0.00+67.42j,0.00-67.42j,"(1.3, 0.5, -0.5)","(1.2, 0.5, -0.5)",0.0+0.0j,0.0+0.0j,0.00-67.42j


In [77]:
display(dfelemcomp[(dfelemcomp.displayed == 0) & (dfelemcomp.summed == 0)])

,"(i,j)","(Szi, Szj)","(Lzi, Lzj)",displayed,summed,BSi,BSj,<LSX>,<LSY>,<LSZ>
0,"[0, 0]","(-0.5, 0.5)","(1.0, 1.0)",0.0+0.0j,0.0+0.0j,"(1.2, 0.5, -0.5)","(1.2, 0.5, 0.5)",0.0+0.0j,0.0+0.0j,0.0+0.0j
5,"[1, 1]","(-0.5, 0.5)","(1.0, 1.0)",0.0+0.0j,0.0+0.0j,"(1.3, 0.5, -0.5)","(1.3, 0.5, 0.5)",0.0+0.0j,0.0+0.0j,0.0+0.0j


In [78]:
print('Comparing eigenvalues from summed matrix with those from displayed matrix')
valsD, vecsD = np.linalg.eigh(SOmat)
valsS, vecsS = np.linalg.eigh(matsum)
dfvals = pd.DataFrame({'summed': valsS, 'displayed': valsD})
dfvals.style.format('{:.2f}')

Comparing eigenvalues from summed matrix with those from displayed matrix


,summed,displayed
0,-134.84,-67.42
1,-0.00,-67.42
2,0.00,67.42
3,134.84,67.42
